In [1]:
from pandas_datareader import data as web
import pandas as pd
import numpy as np
from datetime import datetime as dt
import calendar
import io
import requests
import math
from scipy.optimize import minimize

In [2]:
start_date = ['2017-01-01', '2018-01-01', '2019-01-01', '2020-01-01']
end_date = ['2018-01-01', '2019-01-01', '2020-01-01', '2021-01-01'] 


Tickers = ['MSFT', 'AAPL', 'AMZN', 'TSLA', 'GOOGL', 'GOOG', 'FB', 'JPM', 'HD', 'JNJ', 'UNH', 'PG', 'BAC', 'V', 'ADBE', 'PFE', 'NFLX', 'MA', 'CRM', 'XOM', 'TMO', 'COST', 'CMCSA', 'CSCO', 'AVGO', 'ACN', 'ABT', 'PEP', 'PYPL', 'NKE']
print(Tickers)

['MSFT', 'AAPL', 'AMZN', 'TSLA', 'GOOGL', 'GOOG', 'FB', 'JPM', 'HD', 'JNJ', 'UNH', 'PG', 'BAC', 'V', 'ADBE', 'PFE', 'NFLX', 'MA', 'CRM', 'XOM', 'TMO', 'COST', 'CMCSA', 'CSCO', 'AVGO', 'ACN', 'ABT', 'PEP', 'PYPL', 'NKE']


In [3]:
yBase = 'https://query1.finance.yahoo.com/v7/finance/download/'
yHeaders = {     'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0',     'Accept': 'text/csv;charset=utf-8'     }

def getYahooDf(ticker, startDate, endDate=None): # dates in ISO format
    start = dt.fromisoformat(startDate) # To datetime.datetime object
    fromDate = calendar.timegm(start.utctimetuple()) # To Unix timestamp format used by Yahoo
    if endDate is None:
        end=dt.now()
    else:
        end = dt.fromisoformat(endDate)
    toDate = calendar.timegm(end.utctimetuple())
    params = { 
        'period1': str(fromDate),
        'period2': str(toDate),
        'interval': '1d',
        'events': 'history',
        'includeAdjustedClose': 'true'
    }
    response = requests.request("GET", yBase + ticker, headers=yHeaders, params=params)
    if response.status_code < 200 or response.status_code > 299:
        print("here")
        return None
    else:
        csv = io.StringIO(response.text)
        df = pd.read_csv(csv, index_col='Date')
        return df


In [4]:
df_T = []
for T in range(len(start_date)):
  df = pd.DataFrame()
  for stock in Tickers:
    df[stock] = getYahooDf(stock,start_date[T],end_date[T])['Adj Close']
  df_T.append(df)
print(df_T)

[                 MSFT        AAPL  ...       PYPL        NKE
Date                               ...                      
2017-01-03  58.185513  113.351151  ...  40.250000  49.345898
2017-01-04  57.925186  113.224297  ...  41.000000  50.380665
2017-01-05  57.925186  113.800064  ...  41.060001  50.371166
2017-01-06  58.427261  115.068756  ...  41.450001  51.178093
2017-01-09  58.241299  116.122719  ...  41.400002  50.674961
...               ...         ...  ...        ...        ...
2017-12-22  81.280586  171.474319  ...  73.889999  60.884754
2017-12-26  81.176018  167.124023  ...  74.269997  61.231075
2017-12-27  81.470695  167.153427  ...  74.589996  60.557678
2017-12-28  81.480186  167.623718  ...  74.169998  60.557678
2017-12-29  81.309113  165.811096  ...  73.620003  60.172871

[251 rows x 30 columns],                  MSFT        AAPL  ...       PYPL        NKE
Date                               ...                      
2018-01-02  81.698814  168.779861  ...  73.839996  61.0771

In [5]:
log_returns_T = []

for df in df_T:
  log_returns = np.log(df/df.shift(1))
  log_returns =  log_returns.dropna()
  log_returns_T.append(log_returns)

print(log_returns_T)

[                MSFT      AAPL      AMZN  ...       PEP      PYPL       NKE
Date                                      ...                              
2017-01-04 -0.004484 -0.001120  0.004646  ...  0.001909  0.018462  0.020753
2017-01-05  0.000000  0.005072  0.030270  ... -0.001336  0.001462 -0.000189
2017-01-06  0.008630  0.011087  0.019716  ... -0.001434  0.009453  0.015893
2017-01-09 -0.003188  0.009118  0.001168  ... -0.010576 -0.001207 -0.009880
2017-01-10 -0.000319  0.001008 -0.001281  ... -0.014506 -0.007759 -0.005071
...              ...       ...       ...  ...       ...       ...       ...
2017-12-22  0.000117  0.000000 -0.005463  ...  0.002871 -0.001758 -0.023115
2017-12-26 -0.001287 -0.025697  0.007164  ...  0.002442  0.005130  0.005672
2017-12-27  0.003624  0.000176  0.004663  ...  0.003443  0.004299 -0.011059
2017-12-28  0.000116  0.002810  0.003243  ...  0.000419 -0.005647  0.000000
2017-12-29 -0.002102 -0.010873 -0.014120  ...  0.004764 -0.007443 -0.006375

[250 rows 

In [6]:
Cov_matrix_T = []

for log_returns in log_returns_T:
  Cov_matrix = log_returns.cov()
  Cov_matrix_T.append(Cov_matrix)

print(Cov_matrix_T)

[           MSFT          AAPL      AMZN  ...           PEP      PYPL       NKE
MSFT   0.000086  4.564905e-05  0.000074  ...  9.357483e-06  0.000056  0.000002
AAPL   0.000046  1.231335e-04  0.000074  ...  3.600522e-06  0.000049  0.000013
AMZN   0.000074  7.419914e-05  0.000167  ...  4.879420e-06  0.000065 -0.000004
TSLA   0.000042  6.625823e-05  0.000061  ...  1.300788e-05  0.000050  0.000041
GOOGL  0.000054  5.095989e-05  0.000081  ...  8.962124e-06  0.000063  0.000002
GOOG   0.000056  5.319334e-05  0.000084  ...  9.192497e-06  0.000062  0.000002
FB     0.000054  6.516750e-05  0.000091  ...  2.719186e-06  0.000079  0.000008
JPM    0.000018  2.285113e-05  0.000002  ... -1.396575e-06  0.000007  0.000017
HD     0.000004  5.296865e-06 -0.000002  ...  5.259335e-06  0.000006  0.000016
JNJ    0.000015  3.157480e-06  0.000005  ...  8.843675e-06  0.000013  0.000010
UNH    0.000019  1.950573e-05  0.000021  ...  2.862542e-06  0.000014  0.000013
PG     0.000002  3.096010e-06  0.000002  ...  1.628

In [7]:
mean_returns_T = []

for log_returns in log_returns_T:
  mean_returns = log_returns.mean()
  mean_returns_T.append(mean_returns)

print(mean_returns_T)

[MSFT     0.001338
AAPL     0.001521
AMZN     0.001757
TSLA     0.001444
GOOGL    0.001061
GOOG     0.001144
FB       0.001648
JPM      0.000905
HD       0.001469
JNJ      0.000852
UNH      0.001306
PG       0.000472
BAC      0.001144
V        0.001471
ADBE     0.002107
PFE      0.000527
NFLX     0.001637
MA       0.001478
CRM      0.001484
XOM     -0.000183
TMO      0.001129
COST     0.000817
CMCSA    0.000643
CSCO     0.001046
AVGO     0.001538
ACN      0.001174
ABT      0.001609
PEP      0.000655
PYPL     0.002415
NKE      0.000793
dtype: float64, MSFT     0.000737
AAPL    -0.000337
AMZN     0.000935
TSLA     0.000150
GOOGL   -0.000107
GOOG    -0.000112
FB      -0.001300
JPM     -0.000312
HD      -0.000271
JNJ     -0.000197
UNH      0.000530
PG       0.000194
BAC     -0.000701
V        0.000593
ADBE     0.000966
PFE      0.000863
NFLX     0.001144
MA       0.000888
CRM      0.001086
XOM     -0.000720
TMO      0.000604
COST     0.000356
CMCSA   -0.000660
CSCO     0.000555
AVGO    -0.

In [9]:
T = 4
N = 30
k = 8
X = log_returns_T
Q = Cov_matrix_T
EX = mean_returns_T
E = 0.0012

# Stochastic Gradient descent with layers

In [10]:
def SNNr(t, E, k):

  def softmax(S):  
    S_exp = np.exp(S)
    sums = S_exp.sum(axis = 0)
    pi = S_exp/sums
    return pi

  def onehot(l):
    z = pd.DataFrame(np.zeros(l.shape),columns = X[t].columns)
    for i in range(l.shape[0]):
        z.iloc[i,:] = (l.iloc[i,:] == l.iloc[i,:].max()).astype(float)
    return z

  def minlossindex(l):
    index = np.where(l == np.amin(l))
    return index[0][0]

  ##main funcction

  def grad_update(S,w_tilda,alpha,k):

    for i in range(X[t].shape[0]):
        
        ##forward propagation
        pi = softmax(S)
        u = np.random.uniform(low = 0.0, high = 1.00, size = pi.shape)
        g = -np.log(-np.log(u))
        g = pd.DataFrame(g,columns = X[t].columns)
        gumbel_pi = g + np.log(pi)
        tz = onehot(gumbel_pi)
        z = np.array(tz.sum(axis = 0),ndmin = 2)
        w_prime = np.exp(w_tilda)
        w_bar = np.multiply(w_prime, z)
        w = w_bar/w_bar.sum()
        #print(pi)
        #print(u)
        #print(g)
        #print(gumbel_pi)
        #print(z)
        #print(w_prime)
        #print(w_bar)
        #print(w)
        risk = np.dot(np.dot(z,Q[t]),z.T)[0][0]
        Loss = np.square(np.dot(np.array(X[t].iloc[i,:],ndmin = 2),w.T)[0][0] - E) + risk
        loss_list.append(Loss)
        S_list.append(np.array(S))
        w_tilda_list.append(w_tilda)

        #print(Loss)

        ##gradient calc and descent
        a = np.dot(np.array(X[t].iloc[i,:],ndmin = 2),w.T)[0][0] - E
        b = (1/np.square(w_bar.sum()))
        c = np.multiply(z,w_prime)
        d = np.multiply(w_prime,S - pi)
        
        ##
        first = np.dot(w,Q[t].T+Q[t])
        second = np.dot(w_prime,(S-pi).T)
        third = np.dot(second.T,first)
        risk_grad = third*b
        
        e = np.multiply(first,w_prime)
        
        dw_tilda = a*b*c + b*e
        dS  = a*b*d + risk_grad

        ##update parameter
        w_tilda = w_tilda - (alpha)*(dw_tilda)
        S = S - (alpha)*(dS)

  alphas = [0.1,0.3,1,3,10]
  S = np.random.randn(k,X[t].shape[1])                         
  S = pd.DataFrame(S,columns= X[t].columns)
  w_tilda = np.random.randn(1,X[t].shape[1])

  alpha_losses_list = []        #holds min losses for each alpha
  alpha_S_list = []      #holds min S corresponding to index of min losses
  alpha_w_tilda_list = []       #holds min w_tilda corresponding to index of min losses

  for i in alphas:
    loss_list = []      #holds all losses for each alpha(per iteration)
    S_list = []         #holds all S for each alpha(per iteration)
    w_tilda_list = []    #holds all w_tilda for each alpha(per iteration)
    
    #run model
    grad_update(S,w_tilda,i,k)     
    
    #some updation of lists for optimization
    loss_list = np.array(loss_list,ndmin = 2).T
    w_tilda_list = np.array(w_tilda_list)
    S_list = np.array(S_list)
    ez = minlossindex(loss_list)
    loss_min = loss_list[ez]
    S_min = S_list[ez]
    w_tilda_min = w_tilda_list[ez]
    #print(i)
    alpha_losses_list.append(loss_min[0])
    alpha_S_list.append(S_min)
    alpha_w_tilda_list.append(w_tilda_min)

  ##changing types
  alpha_losses_list = np.array(alpha_losses_list,ndmin= 2)
  alpha_S_list = np.array(alpha_S_list)
  alpha_w_tilda_list = np.array(alpha_w_tilda_list)

  gg = minlossindex(alpha_losses_list)      #index minimum of all the losses in land(of all alphas)
  alpha_losses_list_min = alpha_losses_list[0][gg]           #S corresponding to min loss
  alpha_S_list_min = alpha_S_list[gg]          #w_tilda corresponding to min loss
  alpha_w_tilda_list_min = alpha_w_tilda_list[gg] 

  ## calculate w from feed forwarding
  pi = softmax(alpha_S_list_min)
  pi = pd.DataFrame(np.array(pi),columns = X[t].columns)
  tz = onehot(pi)
  z = np.array(tz.sum(axis = 0),ndmin = 2)
  w_prime = np.exp(alpha_w_tilda_list_min)
  w_bar = np.multiply(w_prime, z)
  w = w_bar/w_bar.sum()
  #print(w)

  return z

In [11]:
Z = SNNr(0,E,k)
Z

array([[1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 1.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]])

In [12]:
# Single-Period MV Model

def optimal_portfolio(N, Q, EX, Er,T):

  def risk(w, Q_t):
    return np.dot(w.T,np.dot(Q_t,w))

  def checkExpectedReturn(w, Er_min, Ex_t):
    res = np.dot(w.T,Ex_t) - Er_min
    return res

  def checkSum(w):
    sum = np.sum(w)
    return sum - 1

  w0 = np.array([1]*N)
  w0 = w0/w0.sum()

  weights = [w0 for i1 in range(T)]

  Ex_return = [Er for j1 in range(T)]

  risk_t = [0 for k1 in range(T)]

  for t in range(T):

    Z = SNNr(t, Ex_return[t], k)

    constraints = [{'type': 'ineq', 'fun': checkExpectedReturn, 'args': (Ex_return[t], EX[t],)},
               {'type': 'eq', 'fun': checkSum}]

    bounds = [(0,1) for i in range(N)]
    for i in range(N):
      if Z[0][i]!=0:
        bounds[i] = (0,1)

      if Z[0][i]==0:
        bounds[i] = (0,0)

    opt = minimize(risk, w0, method='SLSQP', args=(Q[t],), bounds=bounds, constraints=constraints)  # Finding optimal weights for each period

    weights[t] = opt.x

    risk_t[t] = risk(opt.x, Q[t])


  return weights, Ex_return, risk_t


[opt_weights1, Er_t1, r_t1] = optimal_portfolio(N, Q, EX, E, T)
print(opt_weights1)
print(Er_t1)
print(r_t1)

[array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.12089633,
       0.1266706 , 0.11606409, 0.        , 0.        , 0.13684819,
       0.        , 0.        , 0.12884592, 0.12892785, 0.        ,
       0.        , 0.        , 0.        , 0.12335777, 0.        ,
       0.        , 0.        , 0.11838925, 0.        , 0.        ]), array([8.31735172e-17, 0.00000000e+00, 4.27673299e-02, 0.00000000e+00,
       0.00000000e+00, 2.90960989e-11, 0.00000000e+00, 0.00000000e+00,
       2.40645410e-11, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 2.84552914e-11,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       1.87290411e-15, 0.00000000e+00, 2.10636785e-15, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00]), array([0.        , 0.12498734, 0.        , 0.        , 0.

In [13]:
w1 = opt_weights1
risk_t1 = []
Expected_t1 = []
for t in range (T):
  risk = np.dot(w1[t].T, np.dot(Q[t],w1[t]))
  risk_t1.append(risk)
  Et = np.dot(w1[t].T, EX[t])
  Expected_t1.append(Et)

print(risk_t1)
print(Expected_t1)

[2.8287647203527538e-05, 0.0004948122317710495, 9.88270881662067e-05, 0.0011119697747242754]
[0.0011999999999065122, 0.0011257172082362451, 0.0012316845218549827, 0.0012000000023033616]


In [14]:
# Multi-Period MV Model using prospect function

def optimal_portfolio_multi(N, Q, EX, Er, T):

  w0 = np.array([1]*N)
  w0 = w0/w0.sum()

  weights = [w0 for i2 in range(T)]

  Ex_return = [Er for j2 in range(T)]

  risk_t = [0 for k2 in range(T)]

  def risk(w, Q_t):
    return np.dot(w.T,np.dot(Q_t,w))

  def checkExpectedReturn(w, Er_min, Ex_t):
    res = np.dot(w.T,Ex_t) - Er_min
    return res

  def checkSum(w):
    sum = np.sum(w)
    return sum - 1

  Rr = 0.00022    # Expected risk level of investor
  
  def prospectFunc(w, Q_t):  # Calculating the asymmetric investor sentiment
    diff = np.dot(w.T, np.dot(Q_t,w)) - Rr
    a = 0.88
    b = 0.88
    Lambda = 1   # Values of Lambda and Theta depend on the investor sentiments
    Theta = 2
    if diff<0:
      return Lambda*(-1*diff)**a

    else:
      return -1*Theta*(diff)**b

  for t in range(T):

    Z = SNNr(t, Ex_return[t], k)

    constraints = [{'type': 'ineq', 'fun': checkExpectedReturn, 'args': (Ex_return[t], EX[t],)},
               {'type': 'eq', 'fun': checkSum}]

    bounds = [(0,1) for i in range(N)]
    for i in range(N):
      if Z[0][i]!=0:
        bounds[i] = (0,1)

      if Z[0][i]==0:
        bounds[i] = (0,0)

    opt = minimize(risk, weights[t], method='SLSQP', args=(Q[t],), bounds=bounds, constraints=constraints)  # Finding optimal weights for each period

    weights[t] = opt.x

    risk_t[t] = risk(opt.x, Q[t])

    if t<T-1: 
      Ex_return[t+1] = Ex_return[t]*(1 + prospectFunc(weights[t], Q[t]))   # Updating the expected return level for next period using the asymmetric investor sentiment

  return weights, Ex_return, risk_t


[opt_weights, Er_t, r_t] = optimal_portfolio_multi(N, Q, EX, E, T)
print(opt_weights)
print(Er_t)
print(r_t)

[array([0.        , 0.        , 0.12499189, 0.        , 0.        ,
       0.        , 0.12499525, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.12498903,
       0.        , 0.        , 0.        , 0.        , 0.12501479,
       0.        , 0.12500638, 0.        , 0.        , 0.        ,
       0.12500435, 0.        , 0.12501204, 0.12498627, 0.        ]), array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 1.65275789e-20, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 9.15305806e-20,
       8.19175077e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 8.80951843e-20, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.73829377e-20, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00]), array([0.        , 0.12498912, 0.        , 0.1249486 , 0.

In [15]:
w = opt_weights
risk_t = []
Expected_t = []
for t in range (T):
  risk = np.dot(w[t].T, np.dot(Q[t],w[t]))
  risk_t.append(risk)
  Et = np.dot(w[t].T, EX[t])
  Expected_t.append(Et)

print(risk_t)
print(Expected_t)

[3.5760270746538785e-05, 0.0005707389329203218, 0.00010344779362815092, 0.0007252409738461885]
[0.0012988229905380097, 0.0009373472826655459, 0.0012328773453503612, 0.0011988467404352198]
